
# `upstox_v3_18Sep_patched_scalper_llm_router_full.ipynb`  
**NIFTY Intraday Scalper** — Spot‑Anchored + Greeks + IV‑Z + Marketable‑Limit + Recenter + Latency + **LLM Router**

**Scoring router:** Calls a local **stub** first (`LLM_ENDPOINT`), then calls **Ollama** only when `0.35 < score < 0.75`.  
Logs comparative **latency** and **agreement rate** (stub‑only vs final decision).



## Requirements & safety
- Python 3.9+; packages: `pandas`, `numpy`, `upstox-python-sdk` (or `upstox_client`), `requests`, `IPython`
- Set token: `export UPSTOX_ACCESS_TOKEN=...` or paste into `CredentialUpstox.ACCESS_TOKEN`
- Optional local scorer: run `uvicorn score_stub:app --host 127.0.0.1 --port 8000 --workers 1`

**Safety defaults**
- `SIMULATION_MODE = False` (live-first streaming)
- `ORDERS_LIVE = False` (dry‑run orders)
- `EXIT_MANAGER_LIVE = False` (simulated exits)


In [ ]:

# ---- Imports & global config ----
import os, json, time, threading, math, traceback
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
import numpy as np
import pandas as pd
from IPython.display import display

pd.set_option("display.width", 160)
pd.set_option("display.max_columns", 100)

# Upstox SDK
try:
    import upstox_client
    from upstox_client.rest import ApiException
    UPSDK_AVAILABLE = True
except Exception as e:
    UPSDK_AVAILABLE = False
    print("Upstox SDK not available. Install it to run live streaming and orders.")
    print("Example: pip install upstox-python-sdk   # confirm exact name per Upstox docs")


In [ ]:

# ---- Toggles & constants ----
SIMULATION_MODE = False       # Live-first
USE_LLM = True                # Enable scoring gate
ORDERS_LIVE = False           # Default to dry-run
EXIT_MANAGER_LIVE = False     # Simulated exits

UNDERLYING = "NIFTY"
UNDERLYING_SPOT_TOKEN = "NSE_INDEX|Nifty 50"
SPAN_STRIKES = 2              # ATM ± 2 (5 strikes)
WEBSOCKET_MODE = "full_d30"
IST = "Asia/Kolkata"

# Strike steps
STRIKE_STEP = {"NIFTY": 50, "BANKNIFTY": 100, "FINNIFTY": 50}

# Risk limits
MAX_QTY_PER_LEG = 300
MAX_OPEN_LEGS = 6

# Recenter parameters
RECENTER_COOLDOWN_S = 60.0
RECENTER_LOG = True

# === Trade gates & execution params (scalping) ===
DELTA_MIN, DELTA_MAX = 0.45, 0.55           # abs(Delta) in [0.45, 0.55]
SPREAD_MAX = 0.20                            # rupees
DEPTH_IMB_MIN = 0.15                         # direction-of-entry bias
IV_Z_MAX = 2.0                               # skip entries if |z_IV| > 2
IV_Z_MIN_COUNT = 30                          # warm-up samples before gating IV

# Execution (marketable limit orders)
USE_MARKETABLE_LIMITS = True
LIMIT_BUFFER_TICKS = 1                       # 1 tick beyond BBO to ensure fill while capping slippage

# Throttle (burst control)
ORDER_MIN_GAP_MS = 200                       # min time between order sends

# === LLM Scoring ===
# Option A: generic HTTP endpoint that returns {"score": float}
LLM_ENDPOINT = os.getenv("LLM_ENDPOINT", "") # e.g., http://127.0.0.1:8000/score
LLM_TIMEOUT_S = float(os.getenv("LLM_TIMEOUT_S", "0.12"))

# Option B: local Ollama (if OLLAMA_MODEL is set)
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "http://127.0.0.1:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "")  # e.g., "mistral"
OLLAMA_TIMEOUT_S = float(os.getenv("OLLAMA_TIMEOUT_S", "0.20"))
OLLAMA_NUM_PREDICT = int(os.getenv("OLLAMA_NUM_PREDICT", "16"))

LLM_SCORE_THRESHOLD = 0.55                   # final trade/no-trade threshold

# === Router (stub first, then Ollama in gray zone) ===
USE_ROUTER = True
STUB_LOWER, STUB_UPPER = 0.35, 0.75

# Latency logging
LATENCY_LOG = []
LATENCY_LOG_MAX = 5000

# Router logs
ROUTER_LOG = []
ROUTER_LOG_MAX = 5000


In [ ]:

# ---- Credentials & product mapping ----
class CredentialUpstox:
    ACCESS_TOKEN = os.getenv("UPSTOX_ACCESS_TOKEN", "")  # paste here if not using env

PRODUCT_MAP = {
    "MIS": "I",   # Intraday
    "NRML": "D",  # Carry/Delivery
}
DEFAULT_PRODUCT = "MIS"


In [ ]:

# ---- Utilities ----
def to_ist_ms(ms) -> pd.Timestamp:
    try:
        return pd.to_datetime(int(ms), unit="ms", utc=True).tz_convert(IST)
    except Exception:
        return pd.NaT

def resolve_next_listed_expiry(df_instruments: pd.DataFrame, underlying: str, today=None) -> str:
    t = pd.Timestamp.now(IST).normalize() if today is None else pd.Timestamp(today, tz=IST).normalize()
    dfx = df_instruments[
        (df_instruments["segment"] == "NSE_FO") &
        (df_instruments["name"].str.upper() == underlying.upper()) &
        (df_instruments["instrument_type"].isin(["CE","PE"]))
    ].copy()
    if dfx.empty:
        raise ValueError(f"No derivatives found for {underlying} in instruments master")
    dfx["_exp"] = pd.to_datetime(dfx["expiry"], errors="coerce")
    dfx = dfx[dfx["_exp"] >= t.tz_localize(None)]
    if dfx.empty:
        raise ValueError(f"No upcoming expiry >= {t.date()} for {underlying}")
    return dfx["_exp"].min().strftime("%Y-%m-%d")

def nearest_strikes_from_spot(spot_ltp: float, underlying: str, span: int = 2) -> List[int]:
    step = STRIKE_STEP.get(underlying.upper(), 50)
    nearest = int(step * math.floor((spot_ltp + step/2) / step))
    return [nearest + i * step for i in range(-span, span+1)]

def get_upstox_quote_client():
    if not UPSDK_AVAILABLE:
        raise RuntimeError("Upstox SDK is not available.")
    if not CredentialUpstox.ACCESS_TOKEN:
        raise RuntimeError("ACCESS_TOKEN missing. Set UPSTOX_ACCESS_TOKEN or paste in CredentialUpstox.")
    configuration = upstox_client.Configuration()
    configuration.access_token = CredentialUpstox.ACCESS_TOKEN
    return upstox_client.MarketQuoteV3Api(upstox_client.ApiClient(configuration))

def _extract_ltp_from_entry(entry: dict) -> float:
    if not isinstance(entry, dict):
        raise KeyError("Invalid LTP entry")
    for k in ("ltp", "last_price", "last_traded_price", "last", "close"):
        if k in entry and entry[k] is not None:
            return float(entry[k])
    if "ltpc" in entry and isinstance(entry["ltpc"], dict) and "ltp" in entry["ltpc"]:
        return float(entry["ltpc"]["ltp"])
    raise KeyError(f"No LTP field found in entry keys={list(entry.keys())}")

def get_index_spot_ltp(instrument_key: str = None) -> float:
    instrument_key = instrument_key or UNDERLYING_SPOT_TOKEN
    api = get_upstox_quote_client()
    try:
        api_response = api.get_ltp(instrument_key=[instrument_key])
        data_dict = api_response.to_dict() if hasattr(api_response, "to_dict") else dict(api_response)
        data = data_dict.get("data", {})
        entry = data.get(instrument_key) or (next(iter(data.values())) if data else {})
        ltp = _extract_ltp_from_entry(entry)
        if not np.isfinite(ltp):
            raise RuntimeError(f"LTP non-finite: {ltp}")
        return float(ltp)
    except ApiException as e:
        raise RuntimeError(f"Upstox get_ltp ApiException: {e}")


In [ ]:

# ---- Load instruments master (live) ----
def load_instruments_live() -> pd.DataFrame:
    url = "https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz"
    try:
        df = pd.read_json(url)
    except Exception as e:
        raise RuntimeError(f"Failed to load instruments from {url}: {e}")
    # Normalize
    if "expiry" in df:
        exp = pd.to_datetime(df["expiry"], unit="ms", errors="coerce")
        mask = exp.isna() & df["expiry"].notna()
        if mask.any():
            exp2 = pd.to_datetime(df.loc[mask, "expiry"], errors="coerce")
            exp.loc[mask] = exp2
        df["expiry"] = exp.dt.strftime("%Y-%m-%d")
    for col in ("strike_price","lot_size","tick_size","minimum_lot"):
        if col in df:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

df_futureOptions = load_instruments_live()
assert not df_futureOptions.empty, "Instruments master is empty"
display(df_futureOptions.head(6))


In [ ]:

# ---- Build current chain & anchor ATM via spot ----
expiry_target = resolve_next_listed_expiry(df_futureOptions, UNDERLYING)
df_chain = df_futureOptions[
    (df_futureOptions["segment"] == "NSE_FO") &
    (df_futureOptions["name"].str.upper() == UNDERLYING.upper()) &
    (df_futureOptions["instrument_type"].isin(["CE","PE"])) &
    (df_futureOptions["expiry"] == expiry_target)
].copy()
assert not df_chain.empty, f"No options found for {UNDERLYING} expiry={expiry_target}"
print(f"Chain size for {UNDERLYING} expiry {expiry_target}: {len(df_chain)} rows")

# Anchor ATM from live spot LTP (REST) with fallback
try:
    spot_ltp_initial = get_index_spot_ltp(UNDERLYING_SPOT_TOKEN)
    print(f"NIFTY 50 spot LTP: {spot_ltp_initial:.2f}")
except Exception as e:
    print("Spot LTP lookup failed; falling back to chain median. Reason:", e)
    spot_ltp_initial = float(df_chain["strike_price"].median())

strike_list = nearest_strikes_from_spot(spot_ltp_initial, UNDERLYING, span=SPAN_STRIKES)
df_chain_sel = df_chain[df_chain["strike_price"].isin(strike_list)].sort_values(["strike_price","instrument_type"])
token_list = df_chain_sel["instrument_key"].dropna().astype(str).unique().tolist()
assert token_list, "No tokens to subscribe after ATM filtering"

print(f"Selected strikes from spot {spot_ltp_initial:.2f}: {sorted(set(strike_list))}")
display(df_chain_sel.head(10))


In [ ]:

# ---- Feed structures, enrichment, IV stats ----
df_feed = pd.DataFrame(columns=[
    "Token","Ltp","Ltq","Cp",
    "BidP1","BidQ1","AskP1","AskQ1",
    "Ltt","Oi","Iv","Atp","Tbq","Tsq",
    "Delta","Theta","Gamma","Vega","Rho"
])
df_feed_enriched = pd.DataFrame()

_df_lock = threading.Lock()

def enrich_feed(_df_feed: pd.DataFrame, _df_meta: pd.DataFrame, cols_to_add=None) -> pd.DataFrame:
    if cols_to_add is None:
        cols_to_add = ["lot_size","trading_symbol","strike_price","tick_size","instrument_type","expiry","name"]
    left = _df_feed.copy()
    right = _df_meta[["instrument_key"] + [c for c in cols_to_add if c in _df_meta.columns]].drop_duplicates("instrument_key")
    out = left.merge(right, left_on="Token", right_on="instrument_key", how="left", validate="m:1")
    out["Mid"] = np.where(out["BidP1"].notna() & out["AskP1"].notna(), (out["BidP1"] + out["AskP1"])/2.0, out["Ltp"])
    out["Spread"] = np.where(out["BidP1"].notna() & out["AskP1"].notna(), (out["AskP1"] - out["BidP1"]), np.nan)
    out["DepthImb"] = np.where(
        (out["BidQ1"].notna() & out["AskQ1"].notna() & ((out["BidQ1"] + out["AskQ1"]) > 0)),
        (out["BidQ1"] - out["AskQ1"]) / (out["BidQ1"] + out["AskQ1"]),
        np.nan,
    )
    return out

# Online IV z-score stats (per token)
from collections import defaultdict
_iv_stats = defaultdict(lambda: {"n":0, "mean":0.0, "M2":0.0})

def update_iv_stats(token: str, iv_value: float):
    if iv_value is None or not np.isfinite(iv_value): 
        return
    s = _iv_stats[token]
    n1 = s["n"] + 1
    delta = iv_value - s["mean"]
    mean = s["mean"] + delta / n1
    delta2 = iv_value - mean
    M2 = s["M2"] + delta * delta2
    s["n"], s["mean"], s["M2"] = n1, mean, M2

def iv_zscore_for(token: str, iv_value: float):
    s = _iv_stats[token]
    if s["n"] < max(IV_Z_MIN_COUNT, 2):
        return 0.0, True
    var = s["M2"] / max(s["n"] - 1, 1)
    std = math.sqrt(max(var, 1e-12))
    z = (iv_value - s["mean"]) / std if std > 0 else 0.0
    return z, (abs(z) <= IV_Z_MAX)

# Streaming globals
live_streamer = None
current_tokens = list(token_list)
spot_ltp_current = spot_ltp_initial
last_center_nearest = int(STRIKE_STEP[UNDERLYING] * math.floor((spot_ltp_initial + STRIKE_STEP[UNDERLYING]/2)/STRIKE_STEP[UNDERLYING]))
_last_recenter_ts = 0.0


In [ ]:

# ---- Live streamer (Greeks + IV stats + Spot) ----
def start_live_stream(tokens: List[str], mode: str = "full_d30"):
    global live_streamer
    if not UPSDK_AVAILABLE:
        raise RuntimeError("Upstox SDK not installed.")
    if not CredentialUpstox.ACCESS_TOKEN:
        raise RuntimeError("ACCESS_TOKEN missing. Set UPSTOX_ACCESS_TOKEN or paste into CredentialUpstox.ACCESS_TOKEN")

    configuration = upstox_client.Configuration()
    configuration.access_token = CredentialUpstox.ACCESS_TOKEN
    api_client = upstox_client.ApiClient(configuration)
    streamer = upstox_client.MarketDataStreamerV3(api_client, instrument_key=tokens, mode=mode)

    def _on_message(msg):
        global df_feed, df_feed_enriched, spot_ltp_current
        feeds = msg.get("feeds", {})
        for token, payload in feeds.items():
            ff = payload.get("fullFeed",{}).get("marketFF",{})
            ltpc = ff.get("ltpc",{})
            level = ff.get("marketLevel",{}).get("bidAskQuote",[{}])
            greeks = ff.get("optionGreeks",{}) or {}

            if token == UNDERLYING_SPOT_TOKEN:
                try:
                    if ltpc.get("ltp") is not None:
                        spot_ltp_current = float(ltpc.get("ltp"))
                except Exception:
                    pass
                continue

            try:
                iv_val = ff.get("iv")
                if iv_val is not None:
                    update_iv_stats(token, float(iv_val))
            except Exception:
                pass

            row = {
                "Token": token,
                "Ltp": float(ltpc.get("ltp")) if ltpc.get("ltp") is not None else np.nan,
                "Ltq": float(ltpc.get("ltq")) if ltpc.get("ltq") is not None else np.nan,
                "Cp": float(ltpc.get("cp")) if ltpc.get("cp") is not None else np.nan,
                "BidP1": float(level[0].get("bidP")) if level and level[0].get("bidP") is not None else np.nan,
                "BidQ1": float(level[0].get("bidQ")) if level and level[0].get("bidQ") is not None else np.nan,
                "AskP1": float(level[0].get("askP")) if level and level[0].get("askP") is not None else np.nan,
                "AskQ1": float(level[0].get("askQ")) if level and level[0].get("askQ") is not None else np.nan,
                "Ltt": to_ist_ms(ltpc.get("ltt")),
                "Oi": float(ff.get("oi")) if ff.get("oi") is not None else np.nan,
                "Iv": float(ff.get("iv")) if ff.get("iv") is not None else np.nan,
                "Atp": float(ff.get("atp")) if ff.get("atp") is not None else np.nan,
                "Tbq": float(ff.get("tbq")) if ff.get("tbq") is not None else np.nan,
                "Tsq": float(ff.get("tsq")) if ff.get("tsq") is not None else np.nan,
                "Delta": float(greeks.get("delta")) if greeks.get("delta") is not None else np.nan,
                "Theta": float(greeks.get("theta")) if greeks.get("theta") is not None else np.nan,
                "Gamma": float(greeks.get("gamma")) if greeks.get("gamma") is not None else np.nan,
                "Vega":  float(greeks.get("vega"))  if greeks.get("vega")  is not None else np.nan,
                "Rho":   float(greeks.get("rho"))   if greeks.get("rho")   is not None else np.nan,
            }
            with _df_lock:
                if token in df_feed["Token"].values:
                    for k,v in row.items():
                        df_feed.loc[df_feed["Token"]==token, k] = v
                else:
                    df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)
                df_feed_enriched = enrich_feed(df_feed, df_chain)

    def _on_open():
        print("WebSocket opened")

    def _on_error(err):
        print("WebSocket error:", err)

    def _on_close():
        print("WebSocket closed")

    streamer.on_message = _on_message
    streamer.on_open = _on_open
    streamer.on_error = _on_error
    streamer.on_close = _on_close

    streamer.connect()
    live_streamer = streamer
    return streamer

def stop_live_stream():
    global live_streamer
    try:
        if live_streamer is not None:
            if hasattr(live_streamer, "close"):
                live_streamer.close()
            elif hasattr(live_streamer, "disconnect"):
                live_streamer.disconnect()
            elif hasattr(live_streamer, "ws"):
                live_streamer.ws.close()
            print("Stream stop requested.")
    except Exception as e:
        print("Error stopping stream:", e)
    finally:
        live_streamer = None


In [ ]:

# ---- Dynamic recentering ----
def compute_token_list_for_spot(spot_ltp: float) -> List[str]:
    strikes = nearest_strikes_from_spot(spot_ltp, UNDERLYING, span=SPAN_STRIKES)
    df_sel = df_chain[df_chain["strike_price"].isin(strikes)].sort_values(["strike_price","instrument_type"])
    tokens = df_sel["instrument_key"].dropna().astype(str).unique().tolist()
    return tokens

def maybe_recenter_tokens() -> bool:
    global last_center_nearest, _last_recenter_ts, current_tokens, df_chain_sel
    step = STRIKE_STEP.get(UNDERLYING.upper(), 50)
    nearest = int(step * math.floor((spot_ltp_current + step/2)/step))
    now = time.time()
    if abs(nearest - last_center_nearest) >= step and (now - _last_recenter_ts) >= RECENTER_COOLDOWN_S:
        try:
            new_tokens = compute_token_list_for_spot(spot_ltp_current)
            if not new_tokens:
                return False
            tokens_with_spot = list(dict.fromkeys(new_tokens + [UNDERLYING_SPOT_TOKEN]))
            if RECENTER_LOG:
                print(f"[RECENTER] spot={spot_ltp_current:.2f}, nearest={nearest}, old_center={last_center_nearest}")
                print(f"[RECENTER] tokens: {len(current_tokens)} → {len(new_tokens)}")
            stop_live_stream()
            start_live_stream(tokens_with_spot, mode=WEBSOCKET_MODE)
            last_center_nearest = nearest
            _last_recenter_ts = now
            current_tokens = new_tokens
            df_chain_sel = df_chain[df_chain["instrument_key"].isin(new_tokens)].sort_values(["strike_price","instrument_type"])
            return True
        except Exception as e:
            print("Recenter failed:", e)
            traceback.print_exc()
    return False

def recenter_daemon():
    while True:
        try:
            time.sleep(1.0)
            maybe_recenter_tokens()
        except Exception:
            time.sleep(2.0)
            continue


In [ ]:

# ---- Strategy gates & helpers ----
def eligible_entry(row: pd.Series, side: str):
    reasons = []
    delta = row.get("Delta")
    if delta is None or not np.isfinite(delta):
        reasons.append("no_delta")
    else:
        if not (DELTA_MIN <= abs(float(delta)) <= DELTA_MAX):
            reasons.append(f"absΔ={abs(float(delta)):.2f}∉[{DELTA_MIN},{DELTA_MAX}]")
    spread = row.get("Spread")
    if (spread is None) or (not np.isfinite(spread)) or (float(spread) > SPREAD_MAX):
        reasons.append(f"spread={spread} > {SPREAD_MAX}")
    imb = row.get("DepthImb")
    if imb is None or not np.isfinite(imb):
        reasons.append("no_depthimb")
    else:
        imb = float(imb)
        if side.upper() == "BUY" and imb < +DEPTH_IMB_MIN:
            reasons.append(f"imb={imb:.2f} < +{DEPTH_IMB_MIN}")
        if side.upper() == "SELL" and imb > -DEPTH_IMB_MIN:
            reasons.append(f"imb={imb:.2f} > -{DEPTH_IMB_MIN}")
    iv = row.get("Iv")
    z, iv_ok = (0.0, True)
    try:
        if iv is not None and np.isfinite(iv):
            z, iv_ok = iv_zscore_for(row["Token"], float(iv))
            if not iv_ok:
                reasons.append(f"|z_IV|={abs(z):.2f} > {IV_Z_MAX}")
    except Exception:
        pass
    ok = len(reasons) == 0
    return ok, ";".join(reasons), {"iv_z": z}

def _compact_row(r: pd.Series):
    return {
        "token": str(r["Token"]),
        "tsym": str(r.get("trading_symbol", "")),
        "mid": float(r.get("Mid", np.nan)),
        "spread": float(r.get("Spread", np.nan)),
        "delta": float(r.get("Delta", np.nan)),
        "gamma": float(r.get("Gamma", np.nan)),
        "theta": float(r.get("Theta", np.nan)),
        "iv": float(r.get("Iv", np.nan)),
        "depthImb": float(r.get("DepthImb", np.nan)),
        "strike": float(r.get("strike_price", np.nan)),
        "type": str(r.get("instrument_type","")),
    }

def _build_ollama_prompt(context: dict) -> str:
    rules = context.get("rules", {})
    ce, pe = context.get("ce", {}), context.get("pe", {})
    spot = context.get("spot", 0.0)
    return (
        "You are a scalping trade scorer. Return a single JSON object with a numeric field 'score' in [0,1].\n"
        "Higher score means better conditions to open a short ATM strangle now.\n"
        "No explanation, no extra text.\n\n"
        f"spot: {spot}\n"
        f"ce: mid={ce.get('mid')}, spread={ce.get('spread')}, delta={ce.get('delta')}, gamma={ce.get('gamma')}, theta={ce.get('theta')}, iv={ce.get('iv')}, depthImb={ce.get('depthImb')}\n"
        f"pe: mid={pe.get('mid')}, spread={pe.get('spread')}, delta={pe.get('delta')}, gamma={pe.get('gamma')}, theta={pe.get('theta')}, iv={pe.get('iv')}, depthImb={pe.get('depthImb')}\n"
        f"rules: absDelta={rules.get('absDelta')}, spreadMax={rules.get('spreadMax')}, depthImbMin={rules.get('depthImbMin')}, ivZMax={rules.get('ivZMax')}\n\n"
        "Return strictly: {\"score\": <float>}"
    )


In [ ]:

# ---- Gray-zone router scoring ----
def _router_log(entry: dict):
    entry = dict(entry)
    entry.setdefault("t", time.perf_counter())
    ROUTER_LOG.append(entry)
    if len(ROUTER_LOG) > ROUTER_LOG_MAX:
        del ROUTER_LOG[: len(ROUTER_LOG) - ROUTER_LOG_MAX]

def _post_stub(context: dict):
    t0 = time.perf_counter()
    if not LLM_ENDPOINT:
        return 0.6, (time.perf_counter() - t0)*1000.0, False
    try:
        import requests
        r = requests.post(LLM_ENDPOINT, json=context, timeout=LLM_TIMEOUT_S)
        s = float(r.json().get("score", 0.6)) if r.ok else 0.6
        return max(0.0, min(1.0, s)), (time.perf_counter() - t0)*1000.0, True
    except Exception:
        return 0.6, (time.perf_counter() - t0)*1000.0, False

def _post_ollama(context: dict):
    t0 = time.perf_counter()
    if not OLLAMA_MODEL:
        return 0.6, (time.perf_counter() - t0)*1000.0, False
    try:
        import requests, re
        prompt = _build_ollama_prompt(context)
        payload = {
            "model": OLLAMA_MODEL,
            "prompt": prompt,
            "stream": False,
            "options": {"temperature": 0.1, "num_predict": OLLAMA_NUM_PREDICT},
            "format": "json"
        }
        r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=OLLAMA_TIMEOUT_S)
        if not r.ok:
            payload.pop("format", None)
            r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=OLLAMA_TIMEOUT_S)
        txt = r.json().get("response", "") if r.ok else ""
        try:
            s = float(json.loads(txt).get("score", 0.6))
        except Exception:
            m = re.search(r"([01](?:\.\d+)?)", txt)
            s = float(m.group(1)) if m else 0.6
        return max(0.0, min(1.0, s)), (time.perf_counter() - t0)*1000.0, True
    except Exception:
        return 0.6, (time.perf_counter() - t0)*1000.0, False

def _score_with_router(context: dict) -> float:
    thr = LLM_SCORE_THRESHOLD
    # Stub first
    stub_score, stub_ms, stub_ok = _post_stub(context)
    stub_decision = (stub_score >= thr)
    route = "no_stub"
    ollama_score, ollama_ms, used_ollama = (None, 0.0, False)

    if stub_ok:
        if stub_score >= STUB_UPPER:
            final_score = stub_score; route = "stub_high"
        elif stub_score <= STUB_LOWER:
            final_score = stub_score; route = "stub_low"
        else:
            o_score, o_ms, o_ok = _post_ollama(context)
            used_ollama = True; ollama_score, ollama_ms = (o_score, o_ms)
            if o_ok:
                final_score = o_score; route = "gray_ollama"
            else:
                final_score = stub_score; route = "gray_fallback_stub"
    else:
        o_score, o_ms, o_ok = _post_ollama(context)
        used_ollama = True if o_ok else False; ollama_score, ollama_ms = (o_score, o_ms)
        final_score = o_score if o_ok else 0.6
        route = "no_stub_ollama" if o_ok else "neutral"

    final_decision = (final_score >= thr)
    _router_log({
        "route": route,
        "stub_score": stub_score,
        "stub_ms": round(stub_ms, 2),
        "ollama_score": ollama_score,
        "ollama_ms": round(ollama_ms, 2),
        "final_score": final_score,
        "stub_decision": stub_decision,
        "final_decision": final_decision,
        "used_ollama": used_ollama
    })
    return float(final_score)

def router_report(n_tail: int = 200):
    tail = ROUTER_LOG[-n_tail:]
    df = pd.DataFrame(tail)
    if df.empty:
        return df, {}
    agree = (df["stub_decision"] == df["final_decision"]).mean()
    used_ollama_rate = df["used_ollama"].mean()
    avg_stub = df["stub_ms"].mean()
    avg_ollama = df.loc[df["used_ollama"], "ollama_ms"].mean() if (df["used_ollama"].any()) else float("nan")
    routes = df["route"].value_counts().to_dict()
    summary = {
        "n": int(len(df)),
        "agreement_rate": float(round(agree, 4)),
        "used_ollama_rate": float(round(used_ollama_rate, 4)),
        "avg_stub_ms": float(round(avg_stub, 2)),
        "avg_ollama_ms": float(round(avg_ollama, 2)) if avg_ollama == avg_ollama else None,
        "routes": routes
    }
    return df, summary

# Strategy with router-backed scoring
def ask_llm_for_strategy(df_snapshot: pd.DataFrame, use_mock: bool = True) -> Dict[str, Any]:
    if df_snapshot.empty:
        return {"legs": []}
    snap = df_snapshot.dropna(subset=["Mid","strike_price","instrument_type"]).copy()
    if snap.empty:
        return {"legs": []}

    snap["dist"] = (snap["Mid"] - snap["strike_price"]).abs()
    def delta_score(d):
        try:
            d = abs(float(d))
            return abs(0.5 - d) if np.isfinite(d) else 0.5
        except Exception:
            return 0.5
    snap["delta_score"] = snap["Delta"].apply(delta_score)

    picks = []
    for opt in ("CE","PE"):
        sub = snap[snap["instrument_type"] == opt].sort_values(["dist","delta_score"]).head(3)
        if sub.empty:
            continue
        picks.append(sub.iloc[0])
    if len(picks) < 2:
        return {"legs": []}
    ce_row, pe_row = (picks[0], picks[1]) if picks[0]["instrument_type"]=="CE" else (picks[1], picks[0])

    ce_ok, _, _ = eligible_entry(ce_row, side="SELL")
    pe_ok, _, _ = eligible_entry(pe_row, side="SELL")
    if not ce_ok or not pe_ok:
        return {"legs": []}

    context = {
        "spot": float(spot_ltp_current),
        "ce": _compact_row(ce_row),
        "pe": _compact_row(pe_row),
        "position": [],
        "rules": {
            "absDelta": [DELTA_MIN, DELTA_MAX],
            "spreadMax": SPREAD_MAX,
            "depthImbMin": DEPTH_IMB_MIN,
            "ivZMax": IV_Z_MAX
        }
    }
    # Always use router; USE_LLM only controls gating by threshold
    score = _score_with_router(context)
    if USE_LLM and (score < LLM_SCORE_THRESHOLD):
        return {"legs": []}

    legs = []
    for r in (ce_row, pe_row):
        lot = int(r.get("lot_size") or 0) or 50
        legs.append({"token": str(r["Token"]), "side": "SELL", "qty": lot, "product": DEFAULT_PRODUCT, "order_type": "LIMIT"})
    return {"legs": legs, "meta": {"score": score}}


In [ ]:

# ---- Orders, throttle, latency, exits ----
open_positions = {}  # token -> dict(side, qty, avg_price)

def record_fill(fill: Dict[str, Any]):
    token = str(fill["token"])
    side = fill["side"].upper()
    qty = int(fill["qty"])
    price = float(fill.get("price", 0.0))
    pos = open_positions.get(token, {"qty": 0, "side": side, "avg_price": 0.0})
    if pos["qty"] == 0:
        pos = {"qty": qty, "side": side, "avg_price": price}
    else:
        new_qty = pos["qty"] + qty if side == pos["side"] else pos["qty"] - qty
        if new_qty <= 0:
            pos = {"qty": 0, "side": side, "avg_price": 0.0}
        else:
            pos = {"qty": new_qty, "side": side, "avg_price": (pos["avg_price"]*pos["qty"] + price*qty)/max(new_qty,1)}
    open_positions[token] = pos

@dataclass
class ExitConfig:
    dry_run: bool = True
    target_pct: float = 0.2
    stop_pct: float = 0.4
    check_interval_s: float = 1.0

_exit_thread = None
_exit_stop_evt = threading.Event()

def should_exit_position(token: str, row: pd.Series, pos: Dict[str, Any], cfg: ExitConfig) -> Optional[Dict[str, Any]]:
    px = row.get("Mid", np.nan)
    if np.isnan(px):
        px = row.get("Ltp", np.nan)
    if np.isnan(px) or pos["qty"] <= 0:
        return None
    entry = pos["avg_price"]
    side = pos["side"]
    pnl = (px - entry) if side=="BUY" else (entry - px)
    if entry <= 0:
        return None
    pnl_pct = pnl / entry
    if pnl_pct >= cfg.target_pct:
        return {"action":"EXIT","reason":"target","token":token,"qty":pos["qty"],"side": "SELL" if side=="BUY" else "BUY"}
    if pnl_pct <= -cfg.stop_pct:
        return {"action":"EXIT","reason":"stop","token":token,"qty":pos["qty"],"side": "SELL" if side=="BUY" else "BUY"}
    return None

def _exit_worker(cfg: ExitConfig):
    while not _exit_stop_evt.is_set():
        time.sleep(cfg.check_interval_s)
        with _df_lock:
            snapshot = df_feed_enriched.copy()
        for token, pos in list(open_positions.items()):
            if pos["qty"] <= 0:
                continue
            row = snapshot.loc[snapshot["Token"]==token]
            if row.empty:
                continue
            row = row.iloc[0]
            signal = should_exit_position(token, row, pos, cfg)
            if signal:
                if cfg.dry_run:
                    print(f"[EXIT-SIM] {signal}")
                else:
                    print(f"[EXIT-LIVE] Would place exit for {token}: {signal}")
                open_positions[token] = {"qty": 0, "side": pos["side"], "avg_price": pos["avg_price"]}

def start_exit_manager(cfg: ExitConfig):
    global _exit_thread, _exit_stop_evt
    _exit_stop_evt = threading.Event()
    _exit_thread = threading.Thread(target=_exit_worker, args=(cfg,), daemon=True)
    _exit_thread.start()

def stop_exit_manager():
    global _exit_thread, _exit_stop_evt
    _exit_stop_evt.set()
    if _exit_thread is not None:
        _exit_thread.join(timeout=5)

_last_order_ts = 0.0

def _to_tick(price: float, tick: float, side: str):
    if not np.isfinite(price) or not np.isfinite(tick) or tick <= 0:
        return float(price)
    if side.upper() == "BUY":
        return round(math.ceil(price / tick) * tick, 2)
    else:
        return round(math.floor(price / tick) * tick, 2)

def _marketable_limit(row: pd.Series, side: str, buf_ticks: int = LIMIT_BUFFER_TICKS):
    tick = float(row.get("tick_size") or 0.05)
    bid = float(row.get("BidP1") or np.nan)
    ask = float(row.get("AskP1") or np.nan)
    if side.upper() == "BUY":
        base = ask if np.isfinite(ask) else float(row.get("Mid", 0.0))
        return _to_tick(base + buf_ticks*tick, tick, "BUY")
    else:
        base = bid if np.isfinite(bid) else float(row.get("Mid", 0.0))
        px = base - buf_ticks*tick
        return _to_tick(px, tick, "SELL")

def _lat_log(event: str, **kwargs):
    ts = time.perf_counter()
    LATENCY_LOG.append({"t": ts, "event": event, **kwargs})
    if len(LATENCY_LOG) > LATENCY_LOG_MAX:
        del LATENCY_LOG[: len(LATENCY_LOG) - LATENCY_LOG_MAX]

def place_orders(plan: Dict[str, Any], df_enriched: pd.DataFrame, dry_run: bool = True) -> List[Dict[str, Any]]:
    results = []
    if not plan or "legs" not in plan:
        return results

    order_api = None
    if not dry_run:
        if not UPSDK_AVAILABLE:
            raise RuntimeError("Upstox SDK not available for live orders")
        if not CredentialUpstox.ACCESS_TOKEN:
            raise RuntimeError("ACCESS_TOKEN missing for live orders")
        configuration = upstox_client.Configuration()
        configuration.access_token = CredentialUpstox.ACCESS_TOKEN
        api_client = upstox_client.ApiClient(configuration)
        order_api = upstox_client.OrderApi(api_client)

    global _last_order_ts
    for leg in plan["legs"]:
        token = str(leg["token"])
        row = df_enriched.loc[df_enriched["Token"]==token]
        if row.empty:
            results.append({"status":"rejected","reason":"token_not_found","leg":leg})
            continue
        row = row.iloc[0]
        lot = int(row.get("lot_size") or 0)
        qty = int(leg.get("qty", 0))
        if lot and qty % lot != 0:
            results.append({"status":"rejected","reason":f"qty_not_multiple_of_lot({lot})","leg":leg}); continue
        if qty <= 0 or qty > MAX_QTY_PER_LEG:
            results.append({"status":"rejected","reason":"qty_bounds","leg":leg}); continue

        side = leg["side"].upper()
        product_code = PRODUCT_MAP.get(leg.get("product", DEFAULT_PRODUCT), PRODUCT_MAP[DEFAULT_PRODUCT])

        order_type = leg.get("order_type","MARKET").upper()
        px = None
        if USE_MARKETABLE_LIMITS:
            px = _marketable_limit(row, side)
            order_type = "LIMIT"
        else:
            px = float(leg.get("price") or 0.0)

        now = time.perf_counter()
        delay_ms = ORDER_MIN_GAP_MS - (now - _last_order_ts) * 1000.0
        if delay_ms > 0:
            time.sleep(delay_ms / 1000.0)
        _last_order_ts = time.perf_counter()

        _lat_log("order_send", token=token, side=side, qty=qty, px=px, order_type=order_type)
        if dry_run:
            fill = float(row.get("Mid")) if np.isfinite(row.get("Mid", np.nan)) else float(row.get("Ltp", 0.0))
            results.append({"status":"simulated","token":token,"qty":qty,"side":side,"product":product_code,"order_type":order_type,"limit_price":px,"fill_price":fill})
            record_fill({"token": token, "side": side, "qty": qty, "price": fill})
            _lat_log("order_ack", token=token)
        else:
            req = upstox_client.PlaceOrderRequest(
                quantity=str(qty),
                product=product_code,
                validity="DAY",
                price=float(px) if order_type=="LIMIT" else 0.0,
                tag="LLM-STRATEGY",
                instrument_token=token,
                order_type=order_type,
                transaction_type=side,
                disclosed_quantity=0,
                trigger_price=0.0,
                is_amo=False
            )
            resp = order_api.place_order(body=req, api_version="3.0")
            results.append({"status":"placed","order_id":resp.data.order_id,"token":token,"qty":qty,"side":side,"limit_price":px})
            record_fill({"token": token, "side": side, "qty": qty, "price": float(row.get("Mid") or row.get("Ltp") or 0.0)})
            _lat_log("order_ack", token=token)

    return results


In [ ]:

# ---- Orchestration (live-first) ----
if SIMULATION_MODE:
    raise RuntimeError("This notebook is live-first. Set SIMULATION_MODE=False to proceed.")

tokens_with_spot = list(dict.fromkeys(token_list + [UNDERLYING_SPOT_TOKEN]))
streamer = start_live_stream(tokens_with_spot, mode=WEBSOCKET_MODE)

time.sleep(3.0)

with _df_lock:
    snapshot = df_feed_enriched.copy()
print("Feed snapshot rows:", len(snapshot))
display(snapshot.tail(6))

LATENCY_LOG.clear()
def _lat_log(event: str, **kwargs):
    ts = time.perf_counter()
    LATENCY_LOG.append({"t": ts, "event": event, **kwargs})
    if len(LATENCY_LOG) > LATENCY_LOG_MAX:
        del LATENCY_LOG[: len(LATENCY_LOG) - LATENCY_LOG_MAX]

_lat_log("decision_start", rows=len(snapshot))
plan = ask_llm_for_strategy(snapshot, use_mock=True)  # scoring via router; USE_LLM gate applied inside
_lat_log("decision_end", legs=len(plan.get("legs", [])))

try:
    _lat_log("validate_start")
    validated = validate_strategy(plan, snapshot)
    _lat_log("validate_end", ok=True)
except Exception as e:
    validated = None
    _lat_log("validate_end", ok=False, err=str(e))
    print("Validation failed:", e)

orders = []
if validated:
    _lat_log("place_start", nlegs=len(validated["legs"]))
    orders = place_orders(validated, snapshot, dry_run=(not ORDERS_LIVE))
    _lat_log("place_end", norders=len(orders))
    print("Order results:")
    print(json.dumps(orders, indent=2))

cfg = ExitConfig(dry_run=(not EXIT_MANAGER_LIVE))
start_exit_manager(cfg)

_recenter_thread = threading.Thread(target=recenter_daemon, daemon=True)
_recenter_thread.start()

print("Live scalper pipeline running. Use the Stop cell to close sockets and exit manager.")


In [ ]:

# ---- Monitor + diagnostics + router summary ----
with _df_lock:
    mon = df_feed_enriched.copy()
display(mon.tail(12))

if not mon.empty:
    mon["dist"] = (mon["Mid"] - mon["strike_price"]).abs()
    picks = mon.sort_values(["dist"]).groupby("instrument_type").head(2)
    rows = []
    for _, r in picks.iterrows():
        ok_sell, why_sell, ex = eligible_entry(r, side="SELL")
        rows.append({
            "tsym": r.get("trading_symbol",""),
            "type": r.get("instrument_type",""),
            "mid": r.get("Mid"),
            "spread": r.get("Spread"),
            "depthImb": r.get("DepthImb"),
            "delta": r.get("Delta"),
            "iv": r.get("Iv"),
            "ok_sell": ok_sell,
            "why_sell": why_sell,
            "iv_z": ex.get("iv_z")
        })
    df_check = pd.DataFrame(rows)
    display(df_check)
else:
    print("No feed yet.")

# Router summary (agreement & latency)
df_router, summary = router_report(200)
display(df_router.tail(10))
print("Router summary:", summary)


In [ ]:

# ---- Latency report & cleanup ----
def latency_report(n_tail: int = 50) -> pd.DataFrame:
    df = pd.DataFrame(LATENCY_LOG[-n_tail:])
    return df

display(latency_report(30))

# Stop streaming & exit manager
stop_exit_manager()
stop_live_stream()
print("Shutdown requested.")
